In [ ]:
import math
import cmath
import numpy as np
from functools import reduce

class GeneralLogger:
    def __init__(self):
        self.logs = []
        self.step = 0
    def log(self, action, data):
        self.logs.append({'step': self.step, 'action': action, 'data': data})
        self.step += 1
    def clear(self):
        self.logs = []
        self.step = 0
    def print_table(self):
        print("{:<10} {:<30} {:<50}".format("Step", "Action", "Data"))
        print("-" * 90)
        for entry in self.logs:
            print("{:<10} {:<30} {:<50}".format(entry['step'], entry['action'], str(entry['data'])))
    def get_logs(self):
        return self.logs

# Polynomial Operations

In [ ]:
def size_integer(i):
    return math.floor(math.log(i, 2)) + 1 if i > 0 else 1

def size_poly(p):
    return len(p)

def int_div_rem(a, b):
    q = a // b
    r = a % b
    return q, r

In [ ]:
def poly_div_rem(dividend, divisor):
    m = len(dividend) - 1
    n = len(divisor) - 1
    if n < 0:
        raise ValueError("Divisor polynomial cannot be zero.")
    quotient = [0]*(m - n + 1)
    remainder = dividend[:]
    inv_lead = 1 / divisor[-1]
    for i in range(m - n, -1, -1):
        quotient[i] = remainder[i+n] * inv_lead
        for j in range(n+1):
            remainder[i+j] -= quotient[i]*divisor[j]
    while remainder and abs(remainder[-1]) < 1e-12:
        remainder.pop()
    return quotient, remainder

def poly_mul(A, B, n=None):
    deg = (len(A)-1)+(len(B)-1)+1
    if n is None:
        n = deg
    C = [0]*n
    for i in range(len(A)):
        for j in range(len(B)):
            if i+j < n:
                C[i+j] += A[i]*B[j]
    return C

def next_power_of_two(x):
    return 1 << ((x - 1).bit_length())

In [ ]:
def int_to_poly(x, block_size, b, logger=None):
    base = 1 << block_size
    coeff = []
    for _ in range(b):
        coeff.append(x % base)
        x //= base
    if logger:
        logger.log("int_to_poly", f"Converted integer to digits: {coeff} (base {base})")
    return coeff

def poly_to_int(coeff, block_size, logger=None):
    base = 1 << block_size
    carry = 0
    for i in range(len(coeff)):
        total = coeff[i] + carry
        coeff[i] = total % base
        carry = total // base
    if logger:
        logger.log("poly_to_int", f"Normalized coefficients: {coeff} with carry {carry}")
    result = 0
    for i in reversed(range(len(coeff))):
        result = result * base + coeff[i]
    result += carry * (base ** len(coeff))
    return result

In [ ]:
def modinv(a, mod):
    return pow(a, -1, mod)

def fft_mod(a, omega, mod, logger=None, depth=0):
    n = len(a)
    if n == 1:
        if logger:
            logger.log(f"FFT Base (depth {depth})", f"Input: {a}")
        return a[:]
    a_even = fft_mod(a[0::2], (omega*omega) % mod, mod, logger, depth+1)
    a_odd  = fft_mod(a[1::2], (omega*omega) % mod, mod, logger, depth+1)
    y = [0] * n
    omega_k = 1
    for k in range(n//2):
        t = (omega_k * a_odd[k]) % mod
        y[k] = (a_even[k] + t) % mod
        y[k + n//2] = (a_even[k] - t) % mod
        if logger:
            logger.log(f"FFT Depth {depth}", f"k={k}, omega_k={omega_k}, t={t}, y[{k}]={y[k]}, y[{k + n//2}]={y[k + n//2]}")
        omega_k = (omega_k * omega) % mod
    if logger:
         logger.log(f"FFT Combine (depth {depth})", f"y = {y}")
    return y

def ifft_mod(a, omega, mod, logger=None):
    n = len(a)
    omega_inv = modinv(omega, mod)
    y = fft_mod(a, omega_inv, mod, logger)
    n_inv = modinv(n, mod)
    return [(val * n_inv) % mod for val in y]

def poly_mul_mod(p, q, n_fft, mod, logger=None):
    P = p + [0]*(n_fft - len(p))
    Q = q + [0]*(n_fft - len(q))
    if logger:
        logger.log("poly_mul_mod", f"P padded: {P}")
        logger.log("poly_mul_mod", f"Q padded: {Q}")
    g = 3
    omega = pow(g, (mod-1)//n_fft, mod)
    if logger:
        logger.log("poly_mul_mod", f"Using omega = {omega} (n_fft={n_fft}, mod={mod})")
    FP = fft_mod(P, omega, mod, logger)
    FQ = fft_mod(Q, omega, mod, logger)
    Fprod = [(FP[i] * FQ[i]) % mod for i in range(n_fft)]
    if logger:
        logger.log("poly_mul_mod", f"Pointwise product: {Fprod}")
    prod = ifft_mod(Fprod, omega, mod, logger)
    if logger:
        logger.log("poly_mul_mod", f"Convolution result (raw): {prod}")
    return prod

In [ ]:
def schoenhage_strassen(u, v, block_size=4, b=8, mod=65537, logger=None):
    poly_u = int_to_poly(u, block_size, b, logger)
    poly_v = int_to_poly(v, block_size, b, logger)
    n_fft = next_power_of_two(2 * b)
    if logger:
        logger.log("schoenhage_strassen", f"FFT length chosen: {n_fft}")
    prod_poly = poly_mul_mod(poly_u, poly_v, n_fft, mod, logger)
    conv_coeff = prod_poly[:2*b - 1]
    if logger:
        logger.log("schoenhage_strassen", f"Raw convolution coefficients: {conv_coeff}")
    result = poly_to_int(conv_coeff, block_size, logger)
    if logger:
        logger.log("schoenhage_strassen", f"Final integer result: {result}")
    return result

In [ ]:
def integer_reciprocal(P, n, iterations=5, logger=None):
    K = 2 * n
    A = (1 << (K - 1)) // P
    if logger:
        logger.log("Integer Reciprocal", f"Initial A = {A}")
    for _ in range(iterations):
        A_squared = A * A
        prod = A_squared * P
        correction = prod >> (K - 1)
        A = 2 * A - correction
        if logger:
            logger.log("Integer Reciprocal", f"Updated A = {A}")
    return A

def integer_square_via_reciprocal(P, n, logger=None):
    A = (1 << (4*n - 1)) // P
    B = (1 << (4*n - 1)) // (P + 1)
    C = A - B
    D = (1 << (4*n - 1)) // C - P
    if logger:
        logger.log("Integer Squaring", f"A = {A}, B = {B}, C = {C}, D = {D}")
    return D

In [ ]:
def poly_inv(p, n, logger=None):
    if n == 1:
        res = [1 / p[0]]
        if logger:
            logger.log("Poly Inv Base", f"Result: {res}")
        return res
    m = (n + 1) >> 1
    q = poly_inv(p, m, logger)
    prod = poly_mul(p, q, 2*n)
    prod = [1 - coeff for coeff in prod]
    prod[0] += 1
    q_new = poly_mul(q, prod, 2*n)
    if logger:
        logger.log("Poly Inv", f"Intermediate q_new: {q_new}")
    return q_new[:n]

def int_to_mod_residues(u, moduli, logger=None):
    residues = [u % p for p in moduli]
    if logger:
        logger.log("int_to_mod_residues", f"Residues: {residues}")
    return residues

In [ ]:
def chinese_remainder(moduli, residues, logger=None):
    M = reduce(lambda a, b: a * b, moduli)
    total = 0
    for p, r in zip(moduli, residues):
        m_i = M // p
        inv = modinv(m_i, p)
        total += r * m_i * inv
    result = total % M
    if logger:
        logger.log("Chinese Remainder", f"Reconstructed integer: {result}")
    return result

In [ ]:
def poly_mod(p, d):
    _, r = poly_div_rem(p, d)
    return r

def poly_to_mod_residues(u, moduli, logger=None):
    residues = [poly_mod(u, d) for d in moduli]
    if logger:
        logger.log("poly_to_mod_residues", f"Residues: {residues}")
    return residues

def poly_chinese_remainder(moduli, residues):
    def poly_mult(A, B):
        n = len(A) + len(B) - 1
        C = [0]*n
        for i in range(len(A)):
            for j in range(len(B)):
                C[i+j] += A[i]*B[j]
        return C
    P = [1]
    for d in moduli:
        P = poly_mult(P, d)
    total = [0]
    for i in range(len(moduli)):
        c_i = poly_div_rem(P, moduli[i])[0]
        d_i = poly_inv(c_i, len(c_i))
        term = poly_mul(c_i, poly_mul(d_i, residues[i], len(c_i)), len(P))
        total = [ (x+y) for x, y in zip(total + [0]*(len(term)-len(total)), term)]
    return [coeff % 1000000007 for coeff in total]

In [ ]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def extended_gcd(a, b):
    x0, y0 = 1, 0
    x1, y1 = 0, 1
    while b:
        q = a // b
        a, b = b, a - q * b
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
    return a, x0, y0

def gcd_matrix(a, b):
    A, B = a, b
    R = [[1, 0], [0, 1]]
    matrices = []
    while B:
        q = A // B
        r = A % B
        M = [[0, 1], [1, -q]]
        matrices.append(M)
        A, B = B, r
    def mat_mult(X, Y):
        return [[X[0][0]*Y[0][0] + X[0][1]*Y[1][0], X[0][0]*Y[0][1] + X[0][1]*Y[1][1]],
                [X[1][0]*Y[0][0] + X[1][1]*Y[1][0], X[1][0]*Y[0][1] + X[1][1]*Y[1][1]]]
    for M in matrices:
        R = mat_mult(M, R)
    return R

def matrix_extended_euclid(a, b):
    x0, y0 = 1, 0
    x1, y1 = 0, 1
    A, B = a, b
    matrix_list = []
    while B:
        q = A // B
        A, B = B, A - q * B
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
        matrix_list.append([[0, 1], [1, -q]])
    R = [[1, 0], [0, 1]]
    for M in matrix_list:
        R = [[R[0][0]*M[0][0] + R[0][1]*M[1][0], R[0][0]*M[0][1] + R[0][1]*M[1][1]],
             [R[1][0]*M[0][0] + R[1][1]*M[1][0], R[1][0]*M[0][1] + R[1][1]*M[1][1]]]
    return R

## Examples

In [ ]:
logger = GeneralLogger()

u = 123456789
v = 987654321
prod_ss = schoenhage_strassen(u, v, block_size=4, b=8, mod=65537, logger=logger)
print("Schönhage–Strassen product:", prod_ss)
logger.print_table()
logger.clear()

recip = integer_reciprocal(153, 8, iterations=6, logger=logger)
print("Integer reciprocal approximation (floor(2^(15)/153)):", recip)
logger.print_table()
logger.clear()

sq = integer_square_via_reciprocal(13, 4, logger=logger)
print("Integer squaring via reciprocals (13^2):", sq)
logger.print_table()
logger.clear()

p = [4, 1, -3, 2]
n = 4
p_inv = poly_inv(p, n, logger=logger)
print("Polynomial reciprocal (approximate inverse):", p_inv)
logger.print_table()
logger.clear()

moduli = [5, 3, 2]
u_mod = int_to_mod_residues(28, moduli, logger=logger)
print("Modular residues of 28:", u_mod)
reconstructed = chinese_remainder(moduli, u_mod, logger=logger)
print("Reconstructed integer from residues:", reconstructed)
logger.print_table()
logger.clear()

g_val, x_val, y_val = extended_gcd(57, 33)
print("Extended GCD of 57 and 33: gcd =", g_val, "x =", x_val, "y =", y_val)
M_euclid = matrix_extended_euclid(57, 33)
print("Extended Euclid Matrix representation:", M_euclid)

Schönhage–Strassen product: 121932631112635269
Step       Action                         Data                                              
------------------------------------------------------------------------------------------
0          int_to_poly                    Converted integer to digits: [5, 1, 13, 12, 11, 5, 7, 0] (base 16)
1          int_to_poly                    Converted integer to digits: [1, 11, 8, 6, 14, 13, 10, 3] (base 16)
2          schoenhage_strassen            FFT length chosen: 16                             
3          poly_mul_mod                   P padded: [5, 1, 13, 12, 11, 5, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4          poly_mul_mod                   Q padded: [1, 11, 8, 6, 14, 13, 10, 3, 0, 0, 0, 0, 0, 0, 0, 0]
5          poly_mul_mod                   Using omega = 64 (n_fft=16, mod=65537)            
6          FFT Base (depth 4)             Input: [5]                                        
7          FFT Base (depth 4)             Input: [0]          

# Numerical Calculus

In [ ]:
class NumLogger:
    def __init__(self):
        self.logs = []
        self.step = 0
    def log(self, action, data):
        self.logs.append({'step': self.step, 'action': action, 'data': data})
        self.step += 1
    def clear(self):
        self.logs = []
        self.step = 0
    def print_table(self):
        print("{:<10} {:<30} {:<50}".format("Step", "Action", "Data"))
        print("-"*90)
        for entry in self.logs:
            print("{:<10} {:<30} {:<50}".format(entry['step'], entry['action'], str(entry['data'])))
    def get_logs(self):
        return self.logs

In [ ]:
def newton_raphson(f, df, x0, tol=1e-6, max_iter=50, logger=None):
    x = x0
    if logger:
        logger.log("Newton Init", f"x0 = {x0}")
    for n in range(max_iter):
        fx = f(x)
        dfx = df(x)
        if logger:
            logger.log(f"Newton Iter {n}", f"x = {x}, f(x) = {fx}, f'(x) = {dfx}")
        if abs(fx) < tol:
            if logger:
                logger.log("Newton Converged", f"x = {x}")
            return x
        if dfx == 0:
            if logger:
                logger.log("Newton Fail", "Derivative zero")
            return x
        x = x - fx/dfx
    if logger:
        logger.log("Newton MaxIter", f"Final x = {x}")
    return x

In [ ]:
def secant_method(f, x0, x1, tol=1e-6, max_iter=50, logger=None):
    if logger:
        logger.log("Secant Init", f"x0 = {x0}, x1 = {x1}")
    for n in range(max_iter):
        f0 = f(x0)
        f1 = f(x1)
        if logger:
            logger.log(f"Secant Iter {n}", f"x0 = {x0}, x1 = {x1}, f(x0) = {f0}, f(x1) = {f1}")
        if abs(f1) < tol:
            if logger:
                logger.log("Secant Converged", f"x = {x1}")
            return x1
        if f1 - f0 == 0:
            if logger:
                logger.log("Secant Fail", "Zero division possibility")
            return x1
        x2 = x1 - f1*(x1 - x0)/(f1 - f0)
        x0, x1 = x1, x2
    if logger:
        logger.log("Secant MaxIter", f"Final x = {x1}")
    return x1

In [ ]:
def simpsons_rule(f, a, b, n, logger=None):
    h = (b - a) / n
    x_vals = [a + i * h for i in range(n + 1)]
    fx = [f(x) for x in x_vals]
    if logger:
        logger.log("Simpson x", f"{x_vals}")
        logger.log("Simpson f(x)", f"{fx}")
    S = fx[0] + fx[-1] + 4 * sum(fx[i] for i in range(1, n) if i % 2 == 1) + 2 * sum(fx[i] for i in range(2, n) if i % 2 == 0)
    result = h/3 * S
    if logger:
        logger.log("Simpson Result", f"{result}")
    return result

In [ ]:
def gauss_legendre(f, n, logger=None):
    x, w = np.polynomial.legendre.leggauss(n)
    if logger:
        logger.log("Gauss Nodes", f"{x}")
        logger.log("Gauss Weights", f"{w}")
    result = np.dot(w, [f(xi) for xi in x])
    if logger:
        logger.log("Gauss Result", f"{result}")
    return result

In [ ]:
def euler_method(f, x0, y0, h, N, logger=None):
    xs = [x0]
    ys = [y0]
    if logger:
        logger.log("Euler Init", f"x0 = {x0}, y0 = {y0}")
    for i in range(N):
        x = xs[-1]
        y = ys[-1]
        y_new = y + h * f(x, y)
        x_new = x + h
        xs.append(x_new)
        ys.append(y_new)
        if logger:
            logger.log(f"Euler Iter {i}", f"x = {x_new}, y = {y_new}")
    return xs, ys

In [ ]:
def rk4(f, x0, y0, h, N, logger=None):
    xs = [x0]
    ys = [y0]
    if logger:
        logger.log("RK4 Init", f"x0 = {x0}, y0 = {y0}")
    for i in range(N):
        x = xs[-1]
        y = ys[-1]
        k1 = f(x, y)
        k2 = f(x + h/2, y + h/2 * k1)
        k3 = f(x + h/2, y + h/2 * k2)
        k4 = f(x + h, y + h * k3)
        y_new = y + h/6 * (k1 + 2*k2 + 2*k3 + k4)
        x_new = x + h
        xs.append(x_new)
        ys.append(y_new)
        if logger:
            logger.log(f"RK4 Iter {i}", f"x = {x_new}, y = {y_new}, k1 = {k1}, k2 = {k2}, k3 = {k3}, k4 = {k4}")
    return xs, ys

In [ ]:
def finite_difference_first(f, x, h, logger=None):
    result = (f(x + h) - f(x - h)) / (2 * h)
    if logger:
        logger.log("Finite Diff 1st", f"f'({x}) ≈ {result}")
    return result

def finite_difference_second(f, x, h, logger=None):
    result = (f(x + h) - 2 * f(x) + f(x - h)) / (h ** 2)
    if logger:
        logger.log("Finite Diff 2nd", f"f''({x}) ≈ {result}")
    return result

In [ ]:
def richardson_extrapolation(Q_h, Q_h2, p, logger=None):
    result = Q_h2 + (Q_h2 - Q_h) / (2**p - 1)
    if logger:
        logger.log("Richardson", f"R = {result}")
    return result

def conjugate_gradient(A, b, x0, tol=1e-6, max_iter=100, logger=None):
    x = x0
    r = b - A @ x
    p = r.copy()
    rsold = np.dot(r, r)
    if logger:
        logger.log("CG Init", f"x0 = {x0}, r0 = {r}, rsold = {rsold}")
    for i in range(max_iter):
        Ap = A @ p
        alpha = rsold / np.dot(p, Ap)
        x = x + alpha * p
        r = r - alpha * Ap
        rsnew = np.dot(r, r)
        if logger:
            logger.log(f"CG Iter {i}", f"alpha = {alpha}, x = {x}, r = {r}, rsnew = {rsnew}")
        if math.sqrt(rsnew) < tol:
            if logger:
                logger.log("CG Converged", f"x = {x}")
            return x
        p = r + (rsnew/rsold) * p
        rsold = rsnew
    if logger:
        logger.log("CG MaxIter", f"x = {x}")
    return x

## Examples

In [ ]:
logger = NumLogger()

def f_newton(x):
    return x**2 - 2
def df_newton(x):
    return 2*x
nr_result = newton_raphson(f_newton, df_newton, 2.0, tol=1e-6, max_iter=10, logger=logger)
print("Newton–Raphson result for sqrt(2):", nr_result)
logger.print_table()
logger.clear()

sm_result = secant_method(f_newton, 2.0, 1.5, tol=1e-6, max_iter=10, logger=logger)
print("Secant method result for sqrt(2):", sm_result)
logger.print_table()
logger.clear()

simpson_result = simpsons_rule(math.sin, 0, math.pi, 4, logger=logger)
print("Simpson's rule result for ∫_0^π sin(x)dx:", simpson_result)
logger.print_table()
logger.clear()

gauss_result = gauss_legendre(lambda x: x**4, 2, logger=logger)
print("2-point Gauss–Legendre result for ∫_{-1}^1 x^4 dx:", gauss_result)
logger.print_table()
logger.clear()

def ode_f(x, y):
    return x - y
euler_x, euler_y = euler_method(ode_f, 0, 1, 0.25, 4, logger=logger)
print("Euler method result for y'(x)=x-y, y(0)=1:", list(zip(euler_x, euler_y)))
logger.print_table()
logger.clear()

rk4_x, rk4_y = rk4(ode_f, 0, 1, 0.25, 4, logger=logger)
print("RK4 result for y'(x)=x-y, y(0)=1:", list(zip(rk4_x, rk4_y)))
logger.print_table()
logger.clear()

fd1 = finite_difference_first(math.exp, 2, 0.1, logger=logger)
print("Finite difference first derivative of exp at 2:", fd1)
logger.print_table()
logger.clear()

fd2 = finite_difference_second(math.exp, 2, 0.1, logger=logger)
print("Finite difference second derivative of exp at 2:", fd2)
logger.print_table()
logger.clear()

Q_h = 0.5*(1 + math.e)  # Trapezoidal rule with 1 interval for ∫_0^1 e^x dx
def trap_rule(a, b, n):
    h = (b-a)/n
    x_vals = [a + i*h for i in range(n+1)]
    fx = [math.e**x for x in x_vals]
    return h/2*(fx[0] + 2*sum(fx[1:-1]) + fx[-1])
Q_h2 = trap_rule(0, 1, 2)
richardson = richardson_extrapolation(Q_h, Q_h2, 2, logger=logger)
print("Richardson extrapolation result for ∫_0^1 e^x dx:", richardson)
logger.print_table()
logger.clear()

A = np.array([[4, 1], [1, 3]], dtype=float)
b = np.array([1, 2], dtype=float)
x0 = np.zeros(2)
cg_result = conjugate_gradient(A, b, x0, tol=1e-6, max_iter=10, logger=logger)
print("Conjugate Gradient result for Ax=b:", cg_result)
logger.print_table()
logger.clear()

Newton–Raphson result for sqrt(2): 1.4142135623746899
Step       Action                         Data                                              
------------------------------------------------------------------------------------------
0          Newton Init                    x0 = 2.0                                          
1          Newton Iter 0                  x = 2.0, f(x) = 2.0, f'(x) = 4.0                  
2          Newton Iter 1                  x = 1.5, f(x) = 0.25, f'(x) = 3.0                 
3          Newton Iter 2                  x = 1.4166666666666667, f(x) = 0.006944444444444642, f'(x) = 2.8333333333333335
4          Newton Iter 3                  x = 1.4142156862745099, f(x) = 6.007304882871267e-06, f'(x) = 2.8284313725490198
5          Newton Iter 4                  x = 1.4142135623746899, f(x) = 4.510614104447086e-12, f'(x) = 2.8284271247493797
6          Newton Converged               x = 1.4142135623746899                            
Secant method result f